# Vision Transformers: From Theory to Practice

This notebook explores **Vision Transformers (ViT)** through computational analysis and empirical experiments. We'll investigate:

1. **Computational Complexity** - How patch size affects FLOPs, parameters, and memory
2. **Ablation Studies** - The impact of architectural choices (positional embeddings, CLS token)
3. **CIFAR-10 Training** - Practical implementation and performance analysis

**Paper Reference**: "An Image is Worth 16x16 Words: Transformers for Image Recognition at Scale" (Dosovitskiy et al., 2021)

---

## Part 1: Computational Complexity Analysis

### Understanding ViT Computational Trade-offs

Vision Transformers process images as sequences of patches. The patch size directly impacts:
- **Sequence length**: Smaller patches → more tokens → quadratic attention cost
- **Information granularity**: Larger patches → coarser features
- **Model capacity**: Patch embedding parameters vs. transformer parameters

Let's analyze how different patch sizes affect computational complexity.

#### Imports

In [ ]:
import os
import torch
import neural_stack

from neural_stack.utils import model_summary
from neural_stack.models.vision_transformer import VisionTransformer

#### Hyperparameters

In [2]:
IMAGE_SIZE = (32, 32)
NUM_CHANNELS = 3

NUM_LAYERS = 6
NUM_HEADS = 8
EMBED_DIM = 512
PATCH_SIZE = 4
MLP_RATIO = 4

#### Model Configuration

In [3]:
dummy_input = torch.randn((1, NUM_CHANNELS, *IMAGE_SIZE))

for patch_size in [4, 8, 16]:
    vit_model = VisionTransformer(
        img_size=IMAGE_SIZE,
        patch_size=patch_size,
        in_channels=NUM_CHANNELS,
        embed_dim=EMBED_DIM,
        num_heads=NUM_HEADS,
        num_layers=NUM_LAYERS,
        mlp_ratio=MLP_RATIO,
        dropout=0.1,
        num_classes=2
    )

    num_params, num_flops, num_acts, summary = model_summary(vit_model, dummy_input)
    num_patches = IMAGE_SIZE[0] * IMAGE_SIZE[1] // (patch_size ** 2)
    print(f"Patch Size {patch_size}x{patch_size}, Total Patches={num_patches}; #params={num_params}; #flops={num_flops}; #activations={num_acts}")
    print(summary)

    del vit_model

Unsupported operator aten::repeat encountered 1 time(s)
Unsupported operator aten::add encountered 13 time(s)
Unsupported operator aten::div encountered 6 time(s)
Unsupported operator aten::softmax encountered 6 time(s)
Unsupported operator aten::gelu encountered 6 time(s)
Unsupported operator aten::repeat encountered 1 time(s)
Unsupported operator aten::add encountered 13 time(s)
Unsupported operator aten::layer_norm encountered 13 time(s)
Unsupported operator aten::div encountered 6 time(s)
Unsupported operator aten::softmax encountered 6 time(s)
Unsupported operator aten::gelu encountered 6 time(s)
Unsupported operator aten::repeat encountered 1 time(s)
Unsupported operator aten::add encountered 13 time(s)
Unsupported operator aten::div encountered 6 time(s)
Unsupported operator aten::softmax encountered 6 time(s)
Unsupported operator aten::gelu encountered 6 time(s)


Patch Size 4x4, Total Patches=64; #params=18975234; #flops=1256365568; #activations=2232370
| module                                 | #parameters or shape   | #flops     | #activations   |
|:---------------------------------------|:-----------------------|:-----------|:---------------|
| model                                  | 18.975M                | 1.256G     | 2.232M         |
|  patch_embedding                       |  58.88K                |  1.573M    |  32.768K       |
|   patch_embedding.pos_embedding        |   (1, 65, 512)         |            |                |
|   patch_embedding.cls_token            |   (1, 1, 512)          |            |                |
|   patch_embedding.proj                 |   25.088K              |   1.573M   |   32.768K      |
|    patch_embedding.proj.weight         |    (512, 3, 4, 4)      |            |                |
|    patch_embedding.proj.bias           |    (512,)              |            |                |
|  transformer_stack      

Unsupported operator aten::repeat encountered 1 time(s)
Unsupported operator aten::add encountered 13 time(s)
Unsupported operator aten::layer_norm encountered 13 time(s)
Unsupported operator aten::div encountered 6 time(s)
Unsupported operator aten::softmax encountered 6 time(s)
Unsupported operator aten::gelu encountered 6 time(s)
Unsupported operator aten::repeat encountered 1 time(s)
Unsupported operator aten::add encountered 13 time(s)
Unsupported operator aten::div encountered 6 time(s)
Unsupported operator aten::softmax encountered 6 time(s)
Unsupported operator aten::gelu encountered 6 time(s)


Patch Size 8x8, Total Patches=16; #params=19024386; #flops=324738560; #activations=544306
| module                                 | #parameters or shape   | #flops     | #activations   |
|:---------------------------------------|:-----------------------|:-----------|:---------------|
| model                                  | 19.024M                | 0.325G     | 0.544M         |
|  patch_embedding                       |  0.108M                |  1.573M    |  8.192K        |
|   patch_embedding.pos_embedding        |   (1, 17, 512)         |            |                |
|   patch_embedding.cls_token            |   (1, 1, 512)          |            |                |
|   patch_embedding.proj                 |   98.816K              |   1.573M   |   8.192K       |
|    patch_embedding.proj.weight         |    (512, 3, 8, 8)      |            |                |
|    patch_embedding.proj.bias           |    (512,)              |            |                |
|  transformer_stack        

Unsupported operator aten::repeat encountered 1 time(s)
Unsupported operator aten::add encountered 13 time(s)
Unsupported operator aten::layer_norm encountered 13 time(s)
Unsupported operator aten::div encountered 6 time(s)
Unsupported operator aten::softmax encountered 6 time(s)
Unsupported operator aten::gelu encountered 6 time(s)


Patch Size 16x16, Total Patches=4; #params=19313154; #flops=96255488; #activations=156850
| module                                 | #parameters or shape   | #flops     | #activations   |
|:---------------------------------------|:-----------------------|:-----------|:---------------|
| model                                  | 19.313M                | 96.255M    | 0.157M         |
|  patch_embedding                       |  0.397M                |  1.573M    |  2.048K        |
|   patch_embedding.pos_embedding        |   (1, 5, 512)          |            |                |
|   patch_embedding.cls_token            |   (1, 1, 512)          |            |                |
|   patch_embedding.proj                 |   0.394M               |   1.573M   |   2.048K       |
|    patch_embedding.proj.weight         |    (512, 3, 16, 16)    |            |                |
|    patch_embedding.proj.bias           |    (512,)              |            |                |
|  transformer_stack        

### Complexity Report

The table below summarizes the results of the complexity analysis using the above measurements.

| Patch Size | Num Patches | Tokens (w/ CLS) | Attention FLOPs/Layer | Total Params |
|------------|-------------|-----------------|----------------------|--------------|
| 4×4        | 64          | 65              | 72.4 M                    | 18.9 M            |
| 8×8        | 16          | 17              | 18.1 M                    | 19.1 M            |
| 16×16      | 4           | 5               | 5.2 M                    | 19.3            |

The quadratic relationship between the patch size and the number of FLOPs per attention layer is clearly visible: doubling the patch size decreases the number of FLOPs per attention layer by a factor of 4, while the number of parameters remains constant (no parameters in the attention module).

While using larger patches results in a less computationally-intensive model, this comes at the cost of reduced expresivity and level of detail. Depending on the size of the objects of interest in the images, the patch size should be increased enough to capture the desired level of detail. On the other hand, attention compute scales quadratically with the image size as well, so using a larger input resolution has a costly effect on computational complexity, increasing the number of patches. As in everything, a balance must be found.

This can be seen in the original ViT paper, which uses 16x16 patches when training on ImageNet. Models trained on ImageNet commonly use a input resolution of 224x224. This results in 196 total patches. CIFAR10, on the other hand, contains images of size 32x32 -- 7x smaller than ImageNet. This should allow us the use a smaller patch size. In order to get an equivalent number of patches, we would have to set the patch smaller than ~2x2 -- for practical reasons, we shall use a patch size of 4x4.

## Part 2: Ablation Study

---

### Experimental Setup

We'll conduct controlled experiments to understand which components are essential for ViT performance:

**Ablation Experiments**:
1. **Baseline** - Full ViT with positional embeddings and CLS token
2. **No Positional Embedding** - Remove spatial information encoding
3. **No CLS Token** - Use mean pooling instead of learned class token

All models are trained on **CIFAR-10** with identical hyperparameters for fair comparison.

#### Imports

In [3]:
import plotly
import torchvision
import wandb

from tqdm import tqdm
from torchvision import transforms
from torchvision.datasets import CIFAR10

from neural_stack.utils import get_project_root

In [4]:
# Get project root directory
ROOT_DIR = get_project_root()
DATASET_PATH = f"{ROOT_DIR}/data/cifar10"
WANDB_PATH = f"{ROOT_DIR}/data/.wandb"

wandb.login()

wandb: Currently logged in as: andreifurdui (andreifurdui-team) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

#### Dataset Preparation

Load **CIFAR-10** dataset with standard data augmentation:
- Random horizontal flips
- Random cropping with slight scale/ratio variation
- Normalization with dataset statistics

In [5]:
train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomResizedCrop((32, 32), scale=(0.8, 1.0), ratio=(0.9, 1.1)),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

train_dataset = CIFAR10(
    root=DATASET_PATH,
    train=True,
    transform=train_transform,
    download=True
)

test_dataset = CIFAR10(
    root=DATASET_PATH,
    train=False,
    transform=test_transform,
    download=True
)

train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=128,
    shuffle=True,
    num_workers=4,
    pin_memory=True
)

test_loader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=128,
    shuffle=False,
    num_workers=4
)

#### Data Visualization

Sample images from CIFAR-10 to understand the task complexity.

In [6]:
# Visualize some examples
NUM_IMAGES_VIZ = 9
CIFAR_images = torch.stack([test_dataset[idx][0] for idx in range(NUM_IMAGES_VIZ)], dim=0)
img_grid = torchvision.utils.make_grid(CIFAR_images, nrow=3, padding=2)
img_grid = img_grid.permute(1, 2, 0)

# plot the image grid with plotly
fig = plotly.graph_objects.Figure()
fig.add_trace(plotly.graph_objects.Image(z=img_grid.numpy(), zmin=[0,0,0,0], zmax=[1,1,1,1], colormodel="rgb"))
fig.update_layout(width=600, height=600, title="CIFAR-10 Sample Images")
fig.show()

#### Training Configuration

Hyperparameters for all experiments:
- **Architecture**: 4 layers, 8 heads, 256-dim embeddings
- **Patch size**: 8×8 (16 patches from 32×32 images)
- **Optimizer**: AdamW with learning rate 3e-4
- **Training**: 25 epochs with step learning rate decay

In [7]:
IMAGE_SIZE = (32, 32)
NUM_CHANNELS = 3

NUM_LAYERS = 4
NUM_HEADS = 8
EMBED_DIM = 256
PATCH_SIZE = 8
MLP_RATIO = 2

LR = 3e-4
NUM_EPOCHS = 25

device = torch.device("cuda:2" if torch.cuda.is_available() else "cpu")

#### Training & Validation Functions

Define training loop with W&B logging and validation metrics.

In [8]:
def train_epoch(model: torch.nn.Module, dataloader, criterion, optimizer, lr_scheduler, epoch, device, wandb_logger: wandb.Run, print_freq = 50):
    model.train()

    for idx, (img, target) in enumerate(dataloader):
        img = img.to(device)
        target = target.to(device)

        pred = model(img)
        loss = criterion(pred, target)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        accuracy = (pred.argmax(dim=1) == target).float().mean().item()

        wandb_logger.log({
            "train/loss": loss.item(),
            "train/accuracy": accuracy,
            "train/lr": optimizer.param_groups[0]['lr'],
        }, step=epoch * len(dataloader) + idx)
            
    lr_scheduler.step()

def validate(model: torch.nn.Module, dataloader, criterion, device):
    model.eval()

    loss_avg = 0.0
    accuracy_avg = 0.0

    for img, target in dataloader:
        with torch.no_grad():
            img = img.to(device)
            target = target.to(device)

            pred = model(img)
            loss = criterion(pred, target)

            accuracy = (pred.argmax(dim=1) == target).float().mean().item()

            loss_avg += loss.item()
            accuracy_avg += accuracy
    
    loss_avg /= len(dataloader)
    accuracy_avg /= len(dataloader)

    return loss_avg, accuracy_avg

In [11]:
def train(model, train_loader, test_loader, criterion, optimizer, lr_scheduler, device, wandb_logger, num_epochs):
    best_val_accuracy = 0.0

    for epoch in tqdm(range(num_epochs), total=num_epochs):
        train_epoch(
            model=model,
            dataloader=train_loader,
            criterion=criterion,
            optimizer=optimizer,
            lr_scheduler=lr_scheduler,
            epoch=epoch,
            device=device,
            wandb_logger=wandb_logger
        )

        val_loss, val_accuracy = validate(
            model=model,
            dataloader=test_loader,
            criterion=criterion,
            device=device
        )

        print(f"Epoch {epoch+1}/{num_epochs}: Val Loss={val_loss:.4f}, Val Acc={val_accuracy:.4f}")

        if val_accuracy > best_val_accuracy:
            best_val_accuracy = val_accuracy
            torch.save(model.state_dict(), os.path.join(wandb_logger.dir, "best.pth"))
            print(f"New best model saved with Val Acc={best_val_accuracy:.4f}")

        wandb_logger.log({
            "val/loss": val_loss,
            "val/accuracy": val_accuracy,
        }, step=(epoch + 1) * len(train_loader))
    
    wandb_logger.save(os.path.join(wandb_logger.dir, "best.pth"))

---

### Experiment 1: Baseline Model

Train the full ViT model with:
- ✓ Learned positional embeddings
- ✓ CLS token for classification

In [12]:
model = VisionTransformer(
    img_size=IMAGE_SIZE,
    patch_size=PATCH_SIZE,
    in_channels=NUM_CHANNELS,
    embed_dim=EMBED_DIM,
    num_heads=NUM_HEADS,
    num_layers=NUM_LAYERS,
    mlp_ratio=MLP_RATIO,
    dropout=0.1,
    num_classes=10
).to(device)

optimizer = torch.optim.AdamW(
    params=model.parameters(),
    lr=LR
)

lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=12)
criterion = torch.nn.CrossEntropyLoss().to(device)

with wandb.init(project="neural-stack", dir=WANDB_PATH, name="baseline.v2",
     notes="Vision Transformer on CIFAR-10 baseline run",
     group="vision-transformer-cifar10-ablations",
     tags=["vision-transformer", "cifar10"],
     config={
          "num_layers": NUM_LAYERS,
          "num_heads": NUM_HEADS,
          "embed_dim": EMBED_DIM,
          "patch_size": PATCH_SIZE,
          "mlp_ratio": MLP_RATIO,
          "lr": LR,
          }) as wandb_logger:

     train(model, train_loader, test_loader, criterion, optimizer, lr_scheduler, device, wandb_logger, NUM_EPOCHS)
          

  4%|▍         | 1/25 [00:28<11:17, 28.25s/it]

Epoch 1/25: Val Loss=1.4131, Val Acc=0.4939
New best model saved with Val Acc=0.4939


  8%|▊         | 2/25 [00:58<11:18, 29.50s/it]

Epoch 2/25: Val Loss=1.3173, Val Acc=0.5134
New best model saved with Val Acc=0.5134


 12%|█▏        | 3/25 [01:29<11:03, 30.18s/it]

...

 96%|█████████▌| 24/25 [12:41<00:34, 34.16s/it]

Epoch 24/25: Val Loss=0.8994, Val Acc=0.6945
New best model saved with Val Acc=0.6945


100%|██████████| 25/25 [13:16<00:00, 31.85s/it]
wandb: WARNING Saving files without folders. If you want to preserve subdirectories pass base_path to wandb.save, i.e. wandb.save("/mnt/folder/file.h5", base_path="/mnt")
wandb: WARNING Symlinked 1 file into the W&B run directory; call wandb.save again to sync new files.


Epoch 25/25: Val Loss=0.8970, Val Acc=0.6917


train/accuracy,▁▁▁▃▂▃▅▅▅▆▅▅▅▅▅▅▆▄▇▇▇▆▇▇▆▅▆▇█▆▇▇▇▇█▅▆▇▇▇
train/loss,█▆▅▅▆▄▅▄▅▄▄▄▄▄▄▅▃▄▄▄▃▃▃▃▃▂▃▂▂▂▂▂▂▂▂▃▂▁▂▁
train/lr,██████████████████████▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁
val/accuracy,▁▂▃▄▄▅▅▅▆▆▆▆▇▇▇█▇████████
val/loss,█▇▅▅▄▄▃▃▃▃▃▂▁▁▁▁▁▁▁▁▁▁▁▁▁
train/accuracy,0.8125
train/loss,0.54036
train/lr,0.0
val/accuracy,0.69165
val/loss,0.89699


---

### Experiment 2: No Positional Embedding

**Hypothesis**: Without positional embeddings, the model loses spatial awareness and treats patches as an unordered set.

This tests whether ViT can learn spatial relationships purely from data, or if architectural guidance is essential.

In [9]:
model = VisionTransformer(
    img_size=IMAGE_SIZE,
    patch_size=PATCH_SIZE,
    in_channels=NUM_CHANNELS,
    embed_dim=EMBED_DIM,
    num_heads=NUM_HEADS,
    num_layers=NUM_LAYERS,
    mlp_ratio=MLP_RATIO,
    dropout=0.1,
    num_classes=10,
    positional_embedding='none'
).to(device)

optimizer = torch.optim.AdamW(
    params=model.parameters(),
    lr=LR
)

lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=12)
criterion = torch.nn.CrossEntropyLoss().to(device)

with wandb.init(project="neural-stack", dir=WANDB_PATH, name="no-pos-emb",
     notes="Vision Transformer on CIFAR-10 w/o positional embedding run",
     group="vision-transformer-cifar10-ablations",
     tags=["vision-transformer", "cifar10"],
     config={
          "num_layers": NUM_LAYERS,
          "num_heads": NUM_HEADS,
          "embed_dim": EMBED_DIM,
          "patch_size": PATCH_SIZE,
          "mlp_ratio": MLP_RATIO,
          "lr": LR,
          }) as wandb_logger:

     train(model, train_loader, test_loader, criterion, optimizer, lr_scheduler, device, wandb_logger, NUM_EPOCHS)
          

  4%|▍         | 1/25 [00:24<09:53, 24.75s/it]

Epoch 1/25: Val Loss=1.5560, Val Acc=0.4358


  8%|▊         | 2/25 [00:49<09:27, 24.68s/it]

Epoch 2/25: Val Loss=1.4534, Val Acc=0.4717


 12%|█▏        | 3/25 [01:14<09:11, 25.06s/it]

...

 96%|█████████▌| 24/25 [10:11<00:25, 25.64s/it]

Epoch 24/25: Val Loss=1.0211, Val Acc=0.6473


100%|██████████| 25/25 [10:37<00:00, 25.49s/it]

Epoch 25/25: Val Loss=1.0067, Val Acc=0.6517


train/accuracy,▁▁▃▃▃▃▃▃▄▃▄▄▅▃▄▄▄▃▅▅▃▅▅▅▆▆▅▆▅▆▆▇▇▇▆▇▇▇█▇
train/loss,▇▆█▆▆█▆▆▅▆▄▅▄▄▄▅▂▄▃▃▃▃▂▃▃▁▂▃▂▂▄▄▂▂▁▂▁▂▂▁
train/lr,█████████████████▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val/accuracy,▁▂▃▄▅▅▅▅▆▆▆▆▇▇▇▇█████████
val/loss,█▇▇▅▄▄▄▃▃▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁
train/accuracy,0.7125
train/loss,0.80988
train/lr,0.0
val/accuracy,0.6517
val/loss,1.0067


---

### Experiment 3: No CLS Token (Mean Pooling)

**Hypothesis**: The CLS token is a learned aggregation mechanism. Mean pooling provides a simpler alternative.

This tests whether a learnable aggregation token provides advantages over simple averaging.

In [10]:
model = VisionTransformer(
    img_size=IMAGE_SIZE,
    patch_size=PATCH_SIZE,
    in_channels=NUM_CHANNELS,
    embed_dim=EMBED_DIM,
    num_heads=NUM_HEADS,
    num_layers=NUM_LAYERS,
    mlp_ratio=MLP_RATIO,
    dropout=0.1,
    num_classes=10,
    use_cls_token=False
).to(device)

optimizer = torch.optim.AdamW(
    params=model.parameters(),
    lr=LR
)

lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=12)
criterion = torch.nn.CrossEntropyLoss().to(device)

with wandb.init(project="neural-stack", dir=WANDB_PATH, name="no-cls-tok",
     notes="Vision Transformer on CIFAR-10 w/o CLS token (mean pooling) run",
     group="vision-transformer-cifar10-ablations",
     tags=["vision-transformer", "cifar10"],
     config={
          "num_layers": NUM_LAYERS,
          "num_heads": NUM_HEADS,
          "embed_dim": EMBED_DIM,
          "patch_size": PATCH_SIZE,
          "mlp_ratio": MLP_RATIO,
          "lr": LR,
          }) as wandb_logger:

     train(model, train_loader, test_loader, criterion, optimizer, lr_scheduler, device, wandb_logger, NUM_EPOCHS)
          

  4%|▍         | 1/25 [00:26<10:33, 26.39s/it]

Epoch 1/25: Val Loss=1.4229, Val Acc=0.4864


  8%|▊         | 2/25 [00:51<09:55, 25.90s/it]

Epoch 2/25: Val Loss=1.2819, Val Acc=0.5403


 12%|█▏        | 3/25 [01:18<09:32, 26.00s/it]

...

 96%|█████████▌| 24/25 [10:06<00:25, 25.03s/it]

Epoch 24/25: Val Loss=0.8903, Val Acc=0.6978


100%|██████████| 25/25 [10:31<00:00, 25.25s/it]

Epoch 25/25: Val Loss=0.8863, Val Acc=0.6984


train/accuracy,▁▃▄▅▄▆▅▅▅▅▆▅▆▆▆▆▇▆▅▇▆▇▆▇█▇▇███▇▇█▇▇█▇▇█▇
train/loss,▇█▇▆▆▄▄▅▅▄▄▄▄▃▄▂▃▄▂▁▂▂▁▂▃▂▂▂▂▂▃▂▂▁▁▁▂▂▁▁
train/lr,███████████████████▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁
val/accuracy,▁▃▃▄▅▅▅▅▆▆▆▇▇████████████
val/loss,█▆▅▅▄▄▄▃▃▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁
train/accuracy,0.8625
train/loss,0.61381
train/lr,0.0
val/accuracy,0.69838
val/loss,0.88635


### Report & Analysis

The ablation results show the importance of using positional encoding in the ViT model. Model performance drops from 69% to 65% accuracy when positional embedding is not used. 

However, using mean pooling over all the tokens instead of the CLS token does not seem to impact model performance, resulting in similar training and testing metrics.

See the GitHub Issue for loss and metrics curves: https://github.com/andreifurdui/neural-stack/issues/6

Ablation Results Summary Table:
| Model Variant                 | Test Accuracy |
|-------------------------------|---------------|
| Baseline (with CLS & Pos Emb) | 69.5%         |
| No Positional Embedding       | 65.8%         |
| No CLS Token (Mean Pooling)   | 69.8%         |
| Random Guessing               | 10.0%         |

#### The Importance of Positional Embedding

Removing positional embedding essentially removes the model's compass: the patches are treated as a bag-of-tokens and all spatial relation between them being lost after the first Self-Attention layer. In effect, this completely removes the possibility of the ViT architecture to model the inductive biases inherent in a CNN architecture. Obviously, this degrades the model's performance quite significantly, but not completely: information can still be extracted from the contents of the patches, but not by their spatial arrangement. A larger patch size would most probably hurt less from the removal of the positional embedding information, as larger patches would be able to individually capture more information necessary for understanding the contents.

#### 4x4 vs. 16x16

The choice of patch size comes as a compromise between several factors:
- Input size
- Available Compute
- Level of detail

Larger input size would push the patch size up, in order to control the quadratic increase of Attention compute.
Level of detail would push the patch size down, in order to be able to capture more precise features and understand the image at a finer level of detail.

In general, one could safely assume that image size scales with level of detail. However, this is not always true: medical or satelite imagery for example are of very high resolutions, while the level of detail needed to properly extract information from these type of images is quite granular. Domain knowledge serves as the best guide for making this choice.

A hard cap is always given by the available compute, which would put a higher-bound on the computational resources and guide the choice of patch size.

#### CNN vs. ViT Spatial Understanding

CNNs spatial modelling is inherent in its architecture. Overlapping convolutional operations build up a hierarchical understanding of the image. On the other hand, a ViTs spatial modelling must be learned during training. There is no architectural 'help' the ViT can rely on (unless a hand crafted, static positional embedding is used). The transformer architecture must learn spatial information from the data distribution itself.

---

### Additional Experiments

---

### Experiment 4: 2D Positional Embedding

**Hypothesis**: Explicitly modelling the 2-dimensional spatial information through a 2D-aware positional embedding should aid the model in learning spatial relationship.

This tests whether intoducing inductive bias by structurally modelling the positional embedding to describe 2D spatial relationship aids the model in learning.

In [10]:
model = VisionTransformer(
    img_size=IMAGE_SIZE,
    patch_size=PATCH_SIZE,
    in_channels=NUM_CHANNELS,
    embed_dim=EMBED_DIM,
    num_heads=NUM_HEADS,
    num_layers=NUM_LAYERS,
    mlp_ratio=MLP_RATIO,
    dropout=0.1,
    num_classes=10,
    positional_embedding='learned-2d'
).to(device)

optimizer = torch.optim.AdamW(
    params=model.parameters(),
    lr=LR
)

lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=12)
criterion = torch.nn.CrossEntropyLoss().to(device)

with wandb.init(project="neural-stack", dir=WANDB_PATH, name="2d-pos-emb",
     notes="Vision Transformer on CIFAR-10 with 2D positional embedding run",
     group="vision-transformer-cifar10-ablations",
     tags=["vision-transformer", "cifar10"],
     config={
          "num_layers": NUM_LAYERS,
          "num_heads": NUM_HEADS,
          "embed_dim": EMBED_DIM,
          "patch_size": PATCH_SIZE,
          "mlp_ratio": MLP_RATIO,
          "lr": LR,
          }) as wandb_logger:

     train(model, train_loader, test_loader, criterion, optimizer, lr_scheduler, device, wandb_logger, NUM_EPOCHS)
          

  4%|▍         | 1/25 [00:25<10:13, 25.56s/it]

Epoch 1/25: Val Loss=1.3991, Val Acc=0.4986


  8%|▊         | 2/25 [00:50<09:45, 25.44s/it]

Epoch 2/25: Val Loss=1.2725, Val Acc=0.5442


 12%|█▏        | 3/25 [01:16<09:25, 25.71s/it]

...

 96%|█████████▌| 24/25 [10:02<00:25, 25.13s/it]

Epoch 24/25: Val Loss=0.8926, Val Acc=0.6943


100%|██████████| 25/25 [10:28<00:00, 25.14s/it]

Epoch 25/25: Val Loss=0.8861, Val Acc=0.6945


train/accuracy,▁▃▄▂▂▄▃▄▄▅▄▃▅▅▅▆▆▃▄▄▇▇▆▆▆▆▆▇▇▇▇▇▆██▇▆▇██
train/loss,█▅▆▅▅▅▄▆▅▄▃▃▅▃▃▃▂▄▃▂▂▁▂▁▂▂▂▂▁▂▂▂▂▁▂▂▁▂▁▂
train/lr,████████████████████▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁
val/accuracy,▁▃▃▃▄▅▅▅▆▆▆▆▇▇███████████
val/loss,█▆▆▅▄▄▃▃▃▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁
train/accuracy,0.7125
train/loss,0.67633
train/lr,0.0
val/accuracy,0.69452
val/loss,0.88611


#### 2D Positional Embedding Results

Training the ViT with a 2D positional embedding achieves identical results under our configuration, similarly with the results achieved in the paper. This reinforces their conclusion that explicitly modelling the 2D spatial information within the Positional Embedding structure does not bring any improvements to the expressivity of the ViT architecture, which can easily learn 2D spatial dynamics using a 1D positional embedding.

Ablation Results Summary Table:
| Model Variant                 | Test Accuracy |
|-------------------------------|---------------|
| Baseline (with CLS & Pos Emb) | 69.5%         |
| 2D Positional Embedding       | 69.4%         |
| No Positional Embedding       | 65.8%         |
| No CLS Token (Mean Pooling)   | 69.8%         |
| Random Guessing               | 10.0%         |